# Segmenting and clustering neighborhoods in Toronto - assignment
 # Question 1 - method 1

Scraping Wiki tables using BeautifulSoup  

In [52]:
# Create a dataframe of PostalCode, Borough, Neighborhood
#import packages
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd


# wiki url to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')

my_table = soup.find('table', {'class': 'wikitable sortable'})
content = ""
headers = "PostCode, Borough, Neighborhood"

for tr in my_table.find_all('tr'):
    row1 = ""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    content = content+row1[1:]
#print(content)

file=open("toronto.csv", "wb")
file.write(bytes(content, encoding="ascii", errors="ignore"))
df_1b=pd.read_csv("toronto.csv", header=None)
df_1b.columns = ["Neighborhood", "Borough", "Neighborhood"]
df_1b.head()

,Neighborhood,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Question 1 - method 2  (preferred)
 Scraping using pandas method

In [18]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,attrs={"class":"wikitable"})

table = wikitables[0]
table.rename(columns={0: "PostCode", 1:"Borough", 2:"Neighborhood"}, inplace=True)
table.drop([0], axis=0, inplace = True)
table.head()

,PostCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


# Data Cleaning - converting raw dataframe to required table 

In [19]:
df = table
df = df[df.Borough != 'Not assigned']
df.head()

,PostCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [20]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PostCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [24]:
result = df.groupby(['PostCode','Borough'], sort=False).agg(','.join)
df_new = result.reset_index()
df_new.head(12)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


# Display shape of dataframe

In [25]:
df_new.shape

(103, 3)